# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!type publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2025-08	Conveying gender through speech: insights from trans men.	Interspeech	"Focusing on the understudied population of trans men, we present one of the first studies on style shift in trans speakers, considering the phonetic features of trans men's speech in different contexts, and their own beliefs about vocal cues to gender perception."	"Alice Ross, Cliodhna Hughes, Eddie L. Ungless and Catherine Lai. (2025). 'Conveying gender through speech: insights from trans men.' <i>Interspeech 2025</i>, 674-678."	conveying-gender	https://www.isca-archive.org/interspeech_2025/ross25_interspeech.html	
2024-12	Beyond the binary: Limitations and possibilities of gender-related speech technology research.	IEEE SLT	"This paper presents a review of 107 research papers relating to speech and sex or gender in ISCA Interspeech publications between 2013 and 2023. We note the scarcity of work on this topic and find that terminology, par

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2025-08,Conveying gender through speech: insights from...,Interspeech,Focusing on the understudied population of tra...,"Alice Ross, Cliodhna Hughes, Eddie L. Ungless ...",conveying-gender,https://www.isca-archive.org/interspeech_2025/...,NaN
1,2024-12,Beyond the binary: Limitations and possibiliti...,IEEE SLT,This paper presents a review of 107 research p...,"Ariadna Sanchez, Alice Ross, and Nina Markl (2...",beyond-binary-speech-technology,https://ieeexplore.ieee.org/abstract/document/...,NaN
2,2024-06,Is there an uncanny valley for speech? Investi...,Speech Prosody,"In this paper, we aim to describe the relation...","Alice Ross, Martin Corley and Catherine Lai. (...",uncanny-valley-realistic-tts,https://www.isca-archive.org/speechprosody_202...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [8]:
!dir ..\_publications\

 Volume in drive C is OS
 Volume Serial Number is 8A52-B2D5

 Directory of c:\Users\alice\Documents\GitHub\alice-ross.github.io\_publications

29/10/2025  11:57    <DIR>          .
29/10/2025  12:18    <DIR>          ..
29/10/2025  11:11               959 2009-10-01-paper-title-number-1.md
29/10/2025  11:11               897 2010-10-01-paper-title-number-2.md
29/10/2025  11:11               897 2015-10-01-paper-title-number-3.md
29/10/2025  11:11               835 2024-02-17-paper-title-number-4.md
29/10/2025  12:19             1,322 2024-06-uncanny-valley-realistic-tts.md
29/10/2025  12:19             1,748 2024-12-beyond-binary-speech-technology.md
29/10/2025  11:11               699 2025-06-08-paper-title-number-5.md
29/10/2025  12:19             1,301 2025-08-conveying-gender.md
               8 File(s)          8,658 bytes
               2 Dir(s)  697,450,176,512 bytes free


In [10]:
!type ..\_publications\2024-06-uncanny-valley-realistic-tts.md

---
title: "Is there an uncanny valley for speech? Investigating listeners' evaluations of realistic TTS voices."
collection: publications
permalink: /publication/2024-06-uncanny-valley-realistic-tts
excerpt: 'In this paper, we aim to describe the relationship between synthesised voices&apos; perceived human-likeness and pleasantness and seek evidence of auditory uncanny valley effects (UVE) in listeners&apos; evaluations.'
date: 2024-06
venue: 'Speech Prosody'
paperurl: 'https://www.isca-archive.org/speechprosody_2024/ross24_speechprosody.htm'
citation: 'Alice Ross, Martin Corley and Catherine Lai. (2024). &apos;Is there an uncanny valley for speech? Investigating listeners&apos; evaluations of realistic TTS voices.&apos; <i>Speech Prosody 2024</i>, 1115-1119.'
---
In this paper, we aim to describe the relationship between synthesised voices&apos; perceived human-likeness and pleasantness and seek evidence of auditory uncanny valley effects (UVE) in listeners&apos; evaluations.

[Down